In [1]:
from selenium import webdriver 
import pandas as pd
import numpy as np
import requests
import re
from bs4 import BeautifulSoup as BS
import urllib
import time
import json
import random
#from selenium.webdriver.support.wait import WebDriverWait
#from selenium.webdriver.support import expected_conditions as EC
#from selenium.webdriver.common.by import By

In [2]:
df=pd.read_csv('点播信息全.csv',encoding='gbk')
df=df.loc[:38009]

In [3]:
df2=pd.read_csv('电视产品信息数据.csv',encoding='gbk')

In [4]:
df3=df2.loc[df2.loc[:,'分类名称']!='广告'].reset_index(drop=True)

In [ ]:
for i in range(len(df3)):
    if df3.loc[i,'连续剧分类']=='无':
        df3.loc[i,'连续剧分类']=df3.loc[i,'正题名']
        print(i)

In [6]:
rawmlist=df3.loc[:,['连续剧分类','出品年代']]

In [7]:
rawmlist.loc[:,'info']=''
rawmlist.loc[:,'rname']=''
rawmlist.loc[:,'name']=''
rawmlist.loc[:,'url']=''
rawmlist.loc[:,'year']=''
rawmlist.loc[:,'subtype']=''
rawmlist.loc[:,'rate']=''
rawmlist.loc[:,'id']=''
rawmlist.loc[:,'img']=''

In [8]:
rawmlist=rawmlist.drop_duplicates().reset_index(drop=True)

In [28]:
for i in range(1711,len(rawmlist)):
    
    st=rawmlist.loc[i,'连续剧分类']
    
    #print(st)
    st=re.sub('：.*','',st)
    st=re.sub('\(\S+\)','',st)
    st=re.sub('【\S+】','',st)
    st=re.sub('（\S+）','',st)
    st=re.sub('\S+月\S+日','',st)
    st=re.sub('第.季','',st)
    st=re.sub('Ⅱ','',st)
    st=re.sub('Ⅲ','',st)
    #st=re.sub(' ','',st)
    st=st.strip()
    rawmlist.loc[i,'name']=st

In [29]:
rawmlist=rawmlist.drop_duplicates(['name']).reset_index(drop=True)

In [4]:
tvlist=sorted(list(set(mlist)))
len(tvlist)

1522

In [102]:
service_args=[]
service_args.append('--load-images=no')  ##关闭图片加载
service_args.append('--disk-cache=yes')  ##开启缓存
service_args.append('--ignore-ssl-errors=true') ##忽略https错误
dcap={}
dcap["phantomjs.page.settings.userAgent"]='Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.91 Safari/537.36'
driver = webdriver.PhantomJS(executable_path="E:/Applications/Anaconda3/Scripts/phantomjs-2.1.1-windows/bin/phantomjs.exe",service_args=service_args,desired_capabilities=dcap)
driver.implicitly_wait(3)
driver.maximize_window()
#driver.get('https://movie.douban.com/subject_search?search_text=%E5%B0%8F%E7%8C%AA%E4%BD%A9%E5%A5%87&cat=1002')

In [103]:
for i in range(43,len(rawmlist)):
    print(i)
    url='https://movie.douban.com/subject_search?search_text='+urllib.parse.quote(rawmlist.loc[i,'name'].encode('utf8'))+'&cat=1002'
    driver.get(url)
    try:
        WebDriverWait(driver, 5, 0.5).until(lambda driver: driver.find_element_by_id("root"))
        #print(driver.page_source)
    except:
        print('等待超时')
        break
    print('加载完成')
    soup=BS(driver.page_source,'lxml')
    tag=True
    if not soup.find('div',class_='sc-bZQynM'):
        print(i,'没有信息')
        continue
    for s in soup.find_all('div',class_='sc-bZQynM'):
        title=s.find('a',class_="title-text").text
        if re.search('\(\d+\)',title):
            if re.search('\d+',re.search('\(\d+\)',title).group())==rawmlist.loc[i,'出品年代']:
                tag=False
                rawmlist.loc[i,'rname']=title
                rawmlist.loc[i,'url']=s.find('a',class_="title-text")['href']
                rawmlist.loc[i,'info']=s.find('div',class_="meta abstract").text
                break
    if tag:
        s=soup.find('div',class_='sc-bZQynM')
        rawmlist.loc[i,'rname']=s.find('a',class_="title-text").text
        rawmlist.loc[i,'url']=s.find('a',class_="title-text")['href']
        rawmlist.loc[i,'info']=s.find('div',class_="meta abstract").text

43
等待超时


In [138]:
soup=BS(driver.page_source,'lxml')
soup

<html><head></head><body>检测到有异常请求从你的 IP 发出，请 <a href="https://www.douban.com/accounts/login?redir=https%3A%2F%2Fmovie.douban.com%2Fsubject_search%3Fsearch_text%3D%25E6%2588%2591%25E5%25AE%25B6%25E4%25B9%259F%25E6%259C%2589%25E5%25A4%25A7%25E6%2598%258E%25E6%2598%259F%26cat%3D1002">登录</a> 使用豆瓣。
</body></html>

In [3]:
df1=pd.read_csv('用户点播信息.csv',encoding='gbk')

In [5]:
df2=pd.read_csv('用户单片点播信息.csv',encoding='gbk')

In [8]:
mlist=df1.loc[:,"节目名称"].tolist()+df2.loc[:,"影片名称"].tolist()

In [22]:
mlist=list(set(mlist))[1:]
print(mlist)

['飞机总动员2：火线救援(高清)', '奔跑吧兄弟 第二季(04)(高清)', '林海雪原(24)(高清)', '反恐特战队之猎影(42)(高清)', '爱？回家之开心速递(52)', '猜拳(12)(高清)', '太行英雄传(10)(高清)', '06月12日 神奇阿呦', '封神榜(24)(高清)', '虎胆龙威2(高清)', '岁月如金(04)(高清)', '特别呈现：传家本事？他们的一天(高清)', '少林问道(09)(高清)', '巴啦啦小魔仙音符之谜(01)(高清)', '白鹿原(21)(高清)', '同盟(28)(大结局)', '少林问道(08)(高清)', '07月26日 神奇阿呦', '楚乔传(22)(高清)', '特别呈现：微尘中的宇宙(高清)', '早安冬日海(高清)', '不懂撒娇的女人(05)', '黄飞鸿之英雄有梦(高清)(baby裸背，肚兜诱惑)', '憨豆先生动画版 第二季(08)(调整)(高清)', '破晓(43)(高清)', '翻滚吧阿信(高清)(彭于晏大秀肌肉)', '十字路口(高清)', '龙珠传奇(46)(高清)', '糊涂县令郑板桥(33)(高清)', '警察锅哥(31)(高清)', '芈月传奇(04)(高清)', '09月17日 熊出没之环球大冒险(高清)', '谋杀似水年华(高清)', '奔跑吧兄弟 第二季(03)(高清)', '解密(18)', '爱？回家之开心速递(04)', '仙剑奇侠传三(34)(高清)', '憨豆先生动画版 第二季(24)(调整)(高清)', '05月01日 超级飞侠三', '天使的印迹：苦难(高清)', '孤胆保镖(高清)', '证明我爱你(高清)(被论证的情感公式)', '反击(06)(高清)', '神探狄仁杰(05)(高清)', '08月05日 快乐大本营：李宇春“回娘家”送祝福(高清)', '08月06日 我是未来：沈梦辰变身机器侠尬舞(高清)', '一粒红尘(04)(高清)', '火星情报局 第二季(09)(高清)', '怪兽大学(高清)', '特种神枪手(高清)', '外来媳妇本地郎：康婶和她的儿子们(下)', '07月19日 神奇阿呦', '绝代艳后(高清)(媚艳皇后，四处留情)', '自然：深海猎豹(高清)', '赌城群英会(18)', '何所冬暖何所夏凉(09)(高清)',

In [ ]:
for i in range(len(mlist)):
    print(i)
    rawmlist.loc[i+1711,'连续剧分类']=mlist[i]

In [14]:
import json
js=json.loads(res.text)

In [129]:
rawmlist=pd.read_csv('info.csv',encoding='gbk')

In [121]:
rawmlist

,连续剧分类,出品年代,info,rname,name,url,year,subtype,rate,id,img
0,职场是个技术活,2017.0,"[""剧情"", ""爱情""]",职场是个技术活,职场是个技术活,https://movie.douban.com/subject/26108041/,2017.0,tv,"{""stars"": ""25"", ""min"": 0, ""max"": 10, ""average""...",26108041.0,http://img7.doubanio.com/view/photo/s_ratio_po...
1,自然,2017.0,"[""剧情"", ""奇幻"", ""恐怖""]",邪恶力量 第十三季,自然,https://movie.douban.com/subject/26952093/,2017.0,tv,"{""stars"": ""45"", ""min"": 0, ""max"": 10, ""average""...",26952093.0,http://img7.doubanio.com/view/photo/s_ratio_po...
2,深海利剑,2017.0,"[""剧情""]",深海利剑,深海利剑,https://movie.douban.com/subject/26879740/,2017.0,tv,"{""stars"": ""30"", ""min"": 0, ""max"": 10, ""average""...",26879740.0,http://img7.doubanio.com/view/photo/s_ratio_po...
3,金星秀(热),2017.0,"[""脱口秀""]",金星秀,金星秀,https://movie.douban.com/subject/26314504/,2015.0,tv,"{""stars"": ""40"", ""min"": 0, ""max"": 10, ""average""...",26314504.0,http://img3.doubanio.com/view/photo/s_ratio_po...
4,(热)赛小花的远大前程,2017.0,"[""剧情"", ""爱情""]",赛小花的远大前程,赛小花的远大前程,https://movie.douban.com/subject/26772028/,2017.0,tv,"{""stars"": ""30"", ""min"": 0, ""max"": 10, ""average""...",26772028.0,http://img7.doubanio.com/view/photo/s_ratio_po...
5,(热)人民的名义,2017.0,"[""剧情"", ""犯罪"", ""悬疑""]",人民的名义,人民的名义,https://movie.douban.com/subject/26727273/,2017.0,tv,"{""stars"": ""45"", ""min"": 0, ""max"": 10, ""average""...",26727273.0,http://img3.doubanio.com/view/photo/s_ratio_po...
6,(热)择天记,2017.0,"[""剧情"", ""奇幻"", ""古装""]",择天记,择天记,https://movie.douban.com/subject/26614082/,2017.0,tv,"{""stars"": ""20"", ""min"": 0, ""max"": 10, ""average""...",26614082.0,http://img3.doubanio.com/view/photo/s_ratio_po...
7,心理追凶,2016.0,"[""剧情"", ""科幻"", ""犯罪""]",信号,心理追凶,https://movie.douban.com/subject/26310143/,2016.0,tv,"{""stars"": ""45"", ""min"": 0, ""max"": 10, ""average""...",26310143.0,http://img3.doubanio.com/view/photo/s_ratio_po...
8,全职没女,2016.0,"[""剧情"", ""喜剧"", ""爱情""]",全职没女,全职没女,https://movie.douban.com/subject/26726989/,2016.0,tv,"{""stars"": ""40"", ""min"": 0, ""max"": 10, ""average""...",26726989.0,http://img3.doubanio.com/view/photo/s_ratio_po...
9,糊涂县令郑板桥,2016.0,"[""剧情"", ""古装""]",糊涂县令郑板桥,糊涂县令郑板桥,https://movie.douban.com/subject/26365498/,2016.0,tv,"{""stars"": ""30"", ""min"": 0, ""max"": 10, ""average""...",26365498.0,http://img7.doubanio.com/view/photo/s_ratio_po...


In [33]:
ippools=['61.160.212.181:808',
 '183.56.177.130:808',
 '218.72.109.65:18118',
 '183.159.80.54:18118']

In [ ]:
i=2531
way=True
while i<len(rawmlist):
    if i%100==0:
        rawmlist.to_csv('info.csv',index=False)
    time.sleep(random.uniform(1,3))
    print(i)
    
    try:
        if not way:
            raise Exception("way wrong")
        ip=ippools[(i)%(len(ippools))]
        ip=ippools[1]
        proxies={'http':'http://'+ip}
        url="http://api.douban.com//v2/movie/search?q="+rawmlist.loc[i,'name']
        res=requests.get(url,proxies=proxies,timeout=10)
        js=json.loads(res.text)
        way=False
        js["total"]
        way=True
    except Exception as e:
        print(e)
        url="http://api.douban.com//v2/movie/search?q="+rawmlist.loc[i,'name']
        res=requests.get(url)
        js=json.loads(res.text)
    if "total" not in js.keys():
        way=True
        print('请求频率超出',time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time())))
        rawmlist.to_csv('info.csv',index=False)
        time.sleep(300)
        continue
    if js["total"]==0:
        print('没有信息')
        i+=1
        continue
    tag=True
    for s in js["subjects"]:
        if s["title"]==str(rawmlist.loc[i,'name']):
            rawmlist.loc[i,'rname']=s["title"]
            rawmlist.loc[i,'url']=s["alt"]
            rawmlist.loc[i,'info']=json.dumps(s["genres"],ensure_ascii=False)
            rawmlist.loc[i,'rate']=json.dumps(s["rating"],ensure_ascii=False)
            rawmlist.loc[i,'subtype']=s["subtype"]
            rawmlist.loc[i,'year']=s["year"]
            rawmlist.loc[i,'id']=s["id"]
            rawmlist.loc[i,'img']=s["images"]["large"]
            tag=False
            print('名字对应')
            break
        if s["year"]==str(rawmlist.loc[i,'出品年代']):
            rawmlist.loc[i,'rname']=s["title"]
            rawmlist.loc[i,'url']=s["alt"]
            rawmlist.loc[i,'info']=json.dumps(s["genres"],ensure_ascii=False)
            rawmlist.loc[i,'rate']=json.dumps(s["rating"],ensure_ascii=False)
            rawmlist.loc[i,'subtype']=s["subtype"]
            rawmlist.loc[i,'year']=s["year"]
            rawmlist.loc[i,'id']=s["id"]
            rawmlist.loc[i,'img']=s["images"]["large"]
            tag=False
            print('年份对应')
            break
    if tag:
        try:
            s=js["subjects"][0]
        except:
            i+=1; continue
        rawmlist.loc[i,'rname']=s["title"]
        rawmlist.loc[i,'url']=s["alt"]
        rawmlist.loc[i,'info']=json.dumps(s["genres"],ensure_ascii=False)
        rawmlist.loc[i,'rate']=json.dumps(s["rating"],ensure_ascii=False)
        rawmlist.loc[i,'subtype']=s["subtype"]
        rawmlist.loc[i,'year']=s["year"]
        rawmlist.loc[i,'id']=s["id"]
        rawmlist.loc[i,'img']=s["images"]["large"]
    i+=1

In [119]:
rawmlist.to_csv('info.csv',index=False)

In [42]:
rawmlist

,连续剧分类,出品年代,info,rname,name,url,year,subtype,rate,id,img
0,职场是个技术活,2017.0,"[""剧情"", ""爱情""]",职场是个技术活,职场是个技术活,https://movie.douban.com/subject/26108041/,2017,tv,"{""stars"": ""25"", ""min"": 0, ""max"": 10, ""average""...",2.6108e+07,http://img7.doubanio.com/view/photo/s_ratio_po...
1,自然,2017.0,"[""剧情"", ""奇幻"", ""恐怖""]",邪恶力量 第十三季,自然,https://movie.douban.com/subject/26952093/,2017,tv,"{""stars"": ""45"", ""min"": 0, ""max"": 10, ""average""...",2.69521e+07,http://img7.doubanio.com/view/photo/s_ratio_po...
2,深海利剑,2017.0,"[""剧情""]",深海利剑,深海利剑,https://movie.douban.com/subject/26879740/,2017,tv,"{""stars"": ""30"", ""min"": 0, ""max"": 10, ""average""...",2.68797e+07,http://img7.doubanio.com/view/photo/s_ratio_po...
3,金星秀(热),2017.0,"[""脱口秀""]",金星秀,金星秀,https://movie.douban.com/subject/26314504/,2015,tv,"{""stars"": ""40"", ""min"": 0, ""max"": 10, ""average""...",2.63145e+07,http://img3.doubanio.com/view/photo/s_ratio_po...
4,(热)赛小花的远大前程,2017.0,"[""剧情"", ""爱情""]",赛小花的远大前程,赛小花的远大前程,https://movie.douban.com/subject/26772028/,2017,tv,"{""stars"": ""30"", ""min"": 0, ""max"": 10, ""average""...",2.6772e+07,http://img7.doubanio.com/view/photo/s_ratio_po...
5,(热)人民的名义,2017.0,"[""剧情"", ""犯罪"", ""悬疑""]",人民的名义,人民的名义,https://movie.douban.com/subject/26727273/,2017,tv,"{""stars"": ""45"", ""min"": 0, ""max"": 10, ""average""...",2.67273e+07,http://img3.doubanio.com/view/photo/s_ratio_po...
6,(热)择天记,2017.0,"[""剧情"", ""奇幻"", ""古装""]",择天记,择天记,https://movie.douban.com/subject/26614082/,2017,tv,"{""stars"": ""20"", ""min"": 0, ""max"": 10, ""average""...",2.66141e+07,http://img3.doubanio.com/view/photo/s_ratio_po...
7,心理追凶,2016.0,"[""剧情"", ""科幻"", ""犯罪""]",信号,心理追凶,https://movie.douban.com/subject/26310143/,2016,tv,"{""stars"": ""45"", ""min"": 0, ""max"": 10, ""average""...",2.63101e+07,http://img3.doubanio.com/view/photo/s_ratio_po...
8,全职没女,2016.0,"[""剧情"", ""喜剧"", ""爱情""]",全职没女,全职没女,https://movie.douban.com/subject/26726989/,2016,tv,"{""stars"": ""40"", ""min"": 0, ""max"": 10, ""average""...",2.6727e+07,http://img3.doubanio.com/view/photo/s_ratio_po...
9,糊涂县令郑板桥,2016.0,"[""剧情"", ""古装""]",糊涂县令郑板桥,糊涂县令郑板桥,https://movie.douban.com/subject/26365498/,2016,tv,"{""stars"": ""30"", ""min"": 0, ""max"": 10, ""average""...",2.63655e+07,http://img7.doubanio.com/view/photo/s_ratio_po...


In [130]:
rlist=list(set(rawmlist.loc[:,'info'].tolist()))

In [131]:
len(rlist)

472

In [132]:
typelist=[]
for r in rlist[1:]:
    typelist+=json.loads(r)
typelist=list(set(typelist))
print(len(typelist))
print(typelist)

35
['犯罪', '喜剧', '舞台艺术', '新闻', '儿童', '真人秀', '歌舞', '戏曲', '音乐', '冒险', '灾难', '恐怖', '家庭', '短片', '历史', '运动', '动画', '纪录片', '惊悚', '科幻', '战争', '动作', 'Adult', '脱口秀', '悬疑', '剧情', '古装', '鬼怪', '奇幻', '武侠', '同性', '体育', '西部', '传记', '爱情']


In [64]:
rawmlist.fillna('',inplace=True)

In [117]:
rawmlist.loc[rawmlist.loc[:,'info']=='']

,连续剧分类,出品年代,info,rname,name,url,year,subtype,rate,id,img
359,(热)怒海红尘(粤语),2015,,,,,,,,,
653,台湾全攻略,2011,,,台湾全攻略,,,,,,
718,潮语剧场/西游记续,2000,,,潮语剧场/西游记续,,,,,,
731,快乐体验,2017,,,快乐体验,,,,,,
856,银魂JUMP庆典特别篇,2009,,,银魂JUMP庆典特别篇,,,,,,
970,滋味男女(粤语),2015,,,滋味男女,,,,,,
991,完美感觉,2010,,,完美感觉,,,,,,
1047,斗龙战士4双龙核,2016,,,斗龙战士4双龙核,,,,,,
1070,917关晓彤生日会2017,2017,,,917关晓彤生日会2017,,,,,,
1107,测试20170804高清,2017,,,测试20170804高清,,,,,,


In [116]:
for i in range(len(rawmlist)):
    if rawmlist.loc[i,'info']=='':
        if re.search('新闻',rawmlist.loc[i,'name']) or re.search('视角',rawmlist.loc[i,'name']) or re.search('榜样',rawmlist.loc[i,'name']):
            rawmlist.loc[i,'info']=json.dumps(['新闻'])
        if re.search('儿歌',rawmlist.loc[i,'name']) or re.search('宝宝',rawmlist.loc[i,'name']) or re.search('赛尔',rawmlist.loc[i,'name']):
            rawmlist.loc[i,'info']=json.dumps(['儿童','动画'])
        if re.search('CP',rawmlist.loc[i,'name']) or re.search('演唱会',rawmlist.loc[i,'name']) or re.search('音',rawmlist.loc[i,'name']):
            rawmlist.loc[i,'info']=json.dumps(['音乐'])
        if re.search('联赛',rawmlist.loc[i,'name']) or re.search('大奖赛',rawmlist.loc[i,'name']) or re.search('锦标赛',rawmlist.loc[i,'name']) or re.search('世锦赛',rawmlist.loc[i,'name']) or re.search('赛季',rawmlist.loc[i,'name']) or re.search('运动会',rawmlist.loc[i,'name']) or re.search('公开赛',rawmlist.loc[i,'name']) or re.search('全运会',rawmlist.loc[i,'name']) or re.search('亚冠',rawmlist.loc[i,'name']) or re.search('NBA',rawmlist.loc[i,'name']) or re.search('CBA',rawmlist.loc[i,'name'])  or re.search('足',rawmlist.loc[i,'name']) or re.search('球赛',rawmlist.loc[i,'name']) or re.search('冠军杯',rawmlist.loc[i,'name']):
            rawmlist.loc[i,'info']=json.dumps(['体育'])
        if re.search('少儿',rawmlist.loc[i,'name']) or re.search('选举',rawmlist.loc[i,'name']):
            rawmlist.loc[i,'info']=json.dumps(['儿童','家庭','真人秀'])
        if re.search('健康',rawmlist.loc[i,'name']) or re.search('养生',rawmlist.loc[i,'name']):
            rawmlist.loc[i,'info']=json.dumps(['真人秀'])
        if re.search('开心球',rawmlist.loc[i,'name']) or re.search('如意酷宝',rawmlist.loc[i,'name']):
            rawmlist.loc[i,'info']=json.dumps(['儿童','家庭','动画'])
        if re.search('才艺',rawmlist.loc[i,'name']):
            rawmlist.loc[i,'info']=json.dumps(['真人秀'])
        if re.search('甜果',rawmlist.loc[i,'name']) or re.search('大仓库',rawmlist.loc[i,'name']) or re.search('快乐大巴',rawmlist.loc[i,'name']) or re.search('芝麻街',rawmlist.loc[i,'name']):
            rawmlist.loc[i,'info']=json.dumps(['儿童','家庭'])
        if re.search('秀',rawmlist.loc[i,'name']):
            rawmlist.loc[i,'info']=json.dumps(['脱口秀'])
        if re.search('方大同',rawmlist.loc[i,'name']):
            rawmlist.loc[i,'info']=json.dumps(['音乐','真人秀'])
        if re.search('大业',rawmlist.loc[i,'name']):
            rawmlist.loc[i,'info']=json.dumps(['剧情','动作','历史'])
        if re.search('自然',rawmlist.loc[i,'name']) or re.search('地球',rawmlist.loc[i,'name']) or re.search('探索',rawmlist.loc[i,'name']) or re.search('特别呈现',rawmlist.loc[i,'name']) or re.search('建筑',rawmlist.loc[i,'name']):
            rawmlist.loc[i,'info']=json.dumps(['纪录片'])
        if re.search('年级',rawmlist.loc[i,'name']):
            rawmlist.loc[i,'info']=json.dumps(['儿童'])
        if re.search('档案',rawmlist.loc[i,'name']) or re.search('解密',rawmlist.loc[i,'name']):
            rawmlist.loc[i,'info']=json.dumps(['纪录片','历史'])
        if re.search('秦宫冷月',rawmlist.loc[i,'name']):
            rawmlist.loc[i,'info']=json.dumps(['戏曲'])
        if re.search('我是未来',rawmlist.loc[i,'name']):
            rawmlist.loc[i,'info']=json.dumps(['科幻','真人秀','脱口秀'])

In [133]:
sorted(typelist)

['Adult',
 '传记',
 '体育',
 '儿童',
 '冒险',
 '剧情',
 '动作',
 '动画',
 '历史',
 '古装',
 '同性',
 '喜剧',
 '奇幻',
 '家庭',
 '恐怖',
 '悬疑',
 '惊悚',
 '戏曲',
 '战争',
 '新闻',
 '歌舞',
 '武侠',
 '灾难',
 '爱情',
 '犯罪',
 '真人秀',
 '短片',
 '科幻',
 '纪录片',
 '脱口秀',
 '舞台艺术',
 '西部',
 '运动',
 '音乐',
 '鬼怪']